In [1]:
# paquete para leer archivos html
import bs4
# urllib es unpaquete para acceder a contenido de paginas por medio de URL
from urllib.request import urlopen
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

In [2]:
## Funciones auxiliares
def get_soup(url,pars):
    """
    Retorna una sopa de caracteres de la pagina leída en formato especificado
    args:
        url: url de la pagina
        pars: string de tipo de formato en el cual se va a leer la pagina (html o xml)
    returns:
        objeto de tipo BeautifulSoup con su contenido
    """
    #myURL = url
    uClient = urlopen(url)
    page = uClient.read()
    uClient.close()
    page_soup = BeautifulSoup(page,pars)
    return page_soup


def get_opiniones(url_base,tag,atributo,valor,n_pags,conn):
    """
    Consigue todas las opiniones de una URL del foro
    
    args:
        url_base: url base que se concatena con el indice de la pagina
        tag: etiqueta del html donde se encuentra el texto buscado
        atributo: atributo del objeto con etiqueta tag
        valor: valor del atributo
        n_pags: numero maximo de paginas de comentarios maximo a considerar
    
    returns:
        lista con opiniones
    """
    print(url_base)
    opiniones = []
    print("Explorando paginas...")
    for i in range(1,n_pags+1):
        #print("Pagina: ",i)
        URL = url_base + conn + str(i) # "?page=" o 
        soup = get_soup(URL,"html.parser")
        ops_pg = soup.findAll(tag,{atributo:value})
        for op in ops_pg:
            opiniones.append(op.text.replace('\n',''))
        #print("========================================")    
    print("Terminado!")
    return opiniones


## BBVA

In [3]:
url_bbva = "https://opinionescolombia.co/gestion-de-fortunas/bbva-opiniones-reviews"
soup = get_soup(url_bbva,"html.parser")

In [4]:
tag = "div"
atributo = "class"
value = "review-content"
comentarios = soup.findAll(tag,{atributo:value})
opiniones_bbva_col = [] # donde se almacenan todas las opiniones sobre BBVA colombia

for i in range(len(comentarios)):
    opiniones_bbva_col.append(comentarios[i].text)

## Bancolombia

In [5]:
URL_base_bancol = "https://es.trustpilot.com/review/www.bancolombia.com"
tag = "p"
atributo = "class"
value = "review-content__text"
num_pags = 2
opiniones_bancol = get_opiniones(URL_base_bancol, tag, atributo, value, num_pags,"?page=")

https://es.trustpilot.com/review/www.bancolombia.com
Explorando paginas...
Terminado!


## Davivienda

In [7]:
url_dav = "https://appgrooves.com/app/daviplata-global-by-banco-davivienda-sa-1/"
tag = "div"
atributo = "class"
value = "body-content collapse"
num_pags = 4
opiniones_dav = get_opiniones(url_dav, tag, atributo, value, num_pags,"negative?p=")
print("Numero de opiniones conseguidas: ",len(opiniones_dav))

https://appgrooves.com/app/daviplata-global-by-banco-davivienda-sa-1/
Explorando paginas...
Terminado!
Numero de opiniones conseguidas:  39


In [9]:
# limpiando los datos
for i in range(len(opiniones_dav)):
    opiniones_dav[i] = opiniones_dav[i].replace('\t','')

## Pasando informacion a dataframe

In [10]:
opiniones_COL = np.array(opiniones_bbva_col + opiniones_bancol + opiniones_dav)
df_col = pd.DataFrame(opiniones_COL,columns=['opinion'])
df_col['banco'] = np.array(len(opiniones_bbva_col)*["BBVA"]+len(opiniones_bancol)*["Bancolombia"]+len(opiniones_dav)*["Davivienda"])
df_col["pais"] = (len(opiniones_bbva_col)+len(opiniones_bancol)+len(opiniones_dav))*["Colombia"]

In [12]:
# Exportando a excel
df_col.to_excel("opiniones_colombia.xlsx")